In [85]:
import torch
import os
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [ ]:
torch.cuda.get_device_name(0)

In [87]:
path_=os.path.join("../Data")
no_frames=20
no_video=30

In [ ]:
label_map=[name for name in os.listdir(path_) if name != "info.txt" and name != 'data.pth'] #getting label names
print(label_map)

In [ ]:
data=[]
labels=[]
for i,label in enumerate(label_map):
    for personid in os.listdir(os.path.join(path_,label)):
        for video in os.listdir(os.path.join(path_,label,personid)):
            vid=[]
            for frame_no in range(1,no_frames+1):
                res=np.load(os.path.join(path_,label,personid,video,str(frame_no)+"_person.png.npy"))
                vid.append(res)
            data.append(vid)
            labels.append(i)

In [44]:
data=np.array(data)
labels=np.array(labels)

In [ ]:
print(data.shape)
print(len(labels))

In [46]:
labels = F.one_hot(torch.tensor(labels, dtype=torch.int64), num_classes=2)

In [ ]:
print("x shape ",data.shape," y shape",labels.shape)

In [ ]:

x = torch.tensor(data, dtype=torch.float32)  # Convert x to a PyTorch tensor
y = torch.tensor(labels, dtype=torch.float32)  # Ensure y is a PyTorch tensor


dataset = TensorDataset(x, y)


torch.save(dataset, os.path.join(path_,"data.pth"))

print("Dataset saved successfully!")


<h3>Load Data</h3>

In [151]:
loaded_dataset = torch.load(os.path.join(path_,"data.pth"))

data, labels = loaded_dataset[:]
print("x shape:", data.shape)
print("y shape:", labels.shape)


x shape: torch.Size([894, 20, 132])
y shape: torch.Size([894, 2])


C:\Users\abhis\AppData\Local\Temp\ipykernel_17700\3664914814.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dataset = torch.load(os.path.join(path_,"data.pth"))


In [152]:
print(labels.shape,data.shape)


torch.Size([894, 2]) torch.Size([894, 20, 132])


<h3>Model Architecture</h3>

In [153]:
import torch
import torch.nn as nn
import torch.optim as optim

In [154]:
# Define the model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 64, batch_first=True)
        self.lstm2 = nn.LSTM(64, 128, batch_first=True)
        self.lstm3 = nn.LSTM(128, 64, batch_first=True)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.relu(self.fc1(x[:, -1, :]))  # Use the last time step's output
        x = self.relu(self.fc2(x))
        #x = self.softmax(self.fc3(x))
        x = self.fc3(x)
        return x

In [155]:
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [156]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [157]:
input_size = data.shape[2]  # 132
num_classes = labels.shape[1]  # 2
model = LSTMModel(input_size, num_classes).to(device)
model.to(device)

LSTMModel(
  (lstm1): LSTM(132, 64, batch_first=True)
  (lstm2): LSTM(64, 128, batch_first=True)
  (lstm3): LSTM(128, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [158]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Model initialized and moved to device:", device)

Model initialized and moved to device: cuda


In [159]:
num_epochs = 30 

for epoch in range(num_epochs):
    model.train()  
    epoch_loss = 0.0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

      
        outputs = model(batch_x)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))  
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.4f}")


Epoch 1/30, Loss: 0.5906
Epoch 2/30, Loss: 0.4601
Epoch 3/30, Loss: 0.3543
Epoch 4/30, Loss: 0.3205
Epoch 5/30, Loss: 0.3668
Epoch 6/30, Loss: 0.2373
Epoch 7/30, Loss: 0.1836
Epoch 8/30, Loss: 0.1574
Epoch 9/30, Loss: 0.1522
Epoch 10/30, Loss: 0.1521
Epoch 11/30, Loss: 0.1564
Epoch 12/30, Loss: 0.1619
Epoch 13/30, Loss: 0.1476
Epoch 14/30, Loss: 0.1176
Epoch 15/30, Loss: 0.1206
Epoch 16/30, Loss: 0.1280
Epoch 17/30, Loss: 0.1365
Epoch 18/30, Loss: 0.1275
Epoch 19/30, Loss: 0.0881
Epoch 20/30, Loss: 0.1082
Epoch 21/30, Loss: 0.1572
Epoch 22/30, Loss: 0.1177
Epoch 23/30, Loss: 0.0951
Epoch 24/30, Loss: 0.0855
Epoch 25/30, Loss: 0.1100
Epoch 26/30, Loss: 0.0862
Epoch 27/30, Loss: 0.0915
Epoch 28/30, Loss: 0.0815
Epoch 29/30, Loss: 0.0650
Epoch 30/30, Loss: 0.0727


In [166]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        outputs = model(batch_x)
        predictions = torch.argmax(outputs, dim=1)
        label = torch.argmax(batch_y, dim=1)
        total_correct += (predictions == label).sum().item()
        total_samples += label.size(0)

    print(f"Accuracy: {total_correct / total_samples:.4f}")


Accuracy: 0.9553


In [127]:
torch.save(model, "../models/lstm_model_full.pth")

In [167]:
loaded_model = torch.load("../models/lstm_model_full.pth")
loaded_model.to(device)  # Move to the appropriate device if needed
loaded_model.eval()  # Set the model to evaluation mode


C:\Users\abhis\AppData\Local\Temp\ipykernel_17700\6289017.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load("../models/lstm_model_full.pth")


LSTMModel(
  (lstm1): LSTM(132, 64, batch_first=True)
  (lstm2): LSTM(64, 128, batch_first=True)
  (lstm3): LSTM(128, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [168]:
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = loaded_model(batch_x)
        predictions = torch.argmax(outputs, dim=1)
        label = torch.argmax(batch_y, dim=1)
        total_correct += (predictions == label).sum().item()
        total_samples += label.size(0)

    print(f"Accuracy: {total_correct / total_samples:.4f}")

Accuracy: 0.9877


In [169]:
outputs = loaded_model(batch_x)
predictions = torch.argmax(outputs, dim=1)

In [176]:
input_sequence = data[400]  # Replace with your actual input

# Convert NumPy array to PyTorch tensor
input_tensor = torch.tensor(input_sequence, dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension: shape becomes (1, 20, 132)

# Move to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)
model.to(device)

# Perform prediction
with torch.no_grad():
    output = model(input_tensor)  # Output shape: (1, num_classes)
    prediction = torch.argmax(output, dim=1)  # Get the predicted class index

# Print the prediction
print(f"Predicted class: {prediction.item()}")

Predicted class: 0


C:\Users\abhis\AppData\Local\Temp\ipykernel_17700\725162143.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(input_sequence, dtype=torch.float32)
